# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [112]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

geoapify_key

'YOUR KEY HERE'

In [113]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
#city_data_df.head()
geoapify_key

'YOUR KEY HERE'

---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [98]:
%%capture --no-display

# Configure the map plot
map_plot_1 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    size = "Humidity",
    scale = 1,
    color = "City"
)

# Display the map
map_plot_1

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [99]:
# Narrow down cities that fit criteria and drop any results with null values
#ideal_cities = (city_data_df["Max Temp"] >= 65) & (city_data_df["Max Temp"] <= 85)

#ideal_cities = city_data_df.query('Max Temp > 65 and Humidity < 70')
ideal_cities = city_data_df.loc[(city_data_df["Max Temp"] >= 60) 
                                & (city_data_df["Max Temp"] <= 80) 
                                & (city_data_df["Humidity"] <= 60)
                                & (city_data_df["Humidity"] >= 30)
                                & (city_data_df["Cloudiness"] <= 50)
                                & (city_data_df["Wind Speed"] <= 10)]
# Drop any rows with null values
ideal_cities.dropna(inplace = True)

# Display sample data
ideal_cities

C:\Users\nemuk\AppData\Local\Temp\ipykernel_25320\2612717770.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ideal_cities.dropna(inplace = True)


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
138,138,samdrup jongkhar,26.9167,91.6167,69.24,30,0,5.64,BT,1707714253
166,166,sawakin,19.1059,37.3321,67.06,60,22,9.40,SD,1707714263
231,231,whangarei,-35.7317,174.3239,71.85,54,36,5.91,NZ,1707714280
287,287,gisborne,-38.6533,178.0042,63.37,59,5,8.63,NZ,1707714094
299,299,guerrero negro,27.9769,-114.0611,61.16,55,0,7.87,MX,1707714056
322,322,howrah,22.5892,88.3103,73.40,56,20,5.75,IN,1707714058
327,327,north shore,-36.8000,174.7500,75.47,60,33,5.99,NZ,1707714302
344,344,el bauga,18.2620,33.9081,64.69,48,5,8.10,SD,1707714308
371,371,zouerate,22.7187,-12.4521,63.90,49,0,6.64,MR,1707714315
424,424,willowmore,-33.2926,23.4895,66.25,57,12,4.81,ZA,1707714328


### Step 3: Create a new DataFrame called `hotel_df`.

In [100]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_cities[["City", "Country", "Lat", "Lng", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
138,samdrup jongkhar,BT,26.9167,91.6167,30,
166,sawakin,SD,19.1059,37.3321,60,
231,whangarei,NZ,-35.7317,174.3239,54,
287,gisborne,NZ,-38.6533,178.0042,59,
299,guerrero negro,MX,27.9769,-114.0611,55,
322,howrah,IN,22.5892,88.3103,56,
327,north shore,NZ,-36.8000,174.7500,60,
344,el bauga,SD,18.2620,33.9081,48,
371,zouerate,MR,22.7187,-12.4521,49,
424,willowmore,ZA,-33.2926,23.4895,57,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [111]:
# Set parameters to search for a hotel
radius = 10000
params = {'categories':'accommodation.hotel',
         'apiKey':geoapify_key}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lon = row['Lng']
    lat = row['Lat']
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f'circle:{lon},{lat},{radius}'
    params["bias"] = f'proximity:{lon},{lat}'
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
#         # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
#     # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# # Display sample data
hotel_df.head()


Starting hotel search
samdrup jongkhar - nearest hotel: No hotel found
sawakin - nearest hotel: No hotel found
whangarei - nearest hotel: No hotel found
gisborne - nearest hotel: No hotel found
guerrero negro - nearest hotel: No hotel found
howrah - nearest hotel: No hotel found
north shore - nearest hotel: No hotel found
el bauga - nearest hotel: No hotel found
zouerate - nearest hotel: No hotel found
willowmore - nearest hotel: No hotel found
naze - nearest hotel: No hotel found
machali - nearest hotel: No hotel found


,City,Country,Lat,Lng,Humidity,Hotel Name
138,samdrup jongkhar,BT,26.9167,91.6167,30,No hotel found
166,sawakin,SD,19.1059,37.3321,60,No hotel found
231,whangarei,NZ,-35.7317,174.3239,54,No hotel found
287,gisborne,NZ,-38.6533,178.0042,59,No hotel found
299,guerrero negro,MX,27.9769,-114.0611,55,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [108]:
%%capture --no-display

# Configure the map plot
map_plot_2 = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    size = "Humidity",
    scale = 1,
    color = "City",
    hover_cols = ["Hotel Name", "Country"]
)

# Display the map
map_plot_2

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)